# offline data import for josh

In [1]:
# import packages
import os 
path_main = '/home/mcoulter/spykshrk_realtime'
os.chdir(path_main)  # navigate to main spykshrk parent directory 

%load_ext autoreload
%autoreload 2
import matplotlib.pyplot as plt
import loren_frank_data_processing as lfdp
from loren_frank_data_processing import Animal
import numpy as np
import pandas as pd
np.seterr(divide='ignore',invalid='ignore')
import trodes2SS
import scipy as sp
import sungod_util
from spykshrk.franklab.data_containers import RippleTimes, pos_col_format, FlatLinearPosition, SpikeFeatures
#FlatLinearPosition, SpikeFeatures, Posteriors, \
         #EncodeSettings, pos_col_format, SpikeObservation, RippleTimes, DayEpochEvent, DayEpochTimeSeries
from spykshrk.franklab.pp_decoder.pp_clusterless import OfflinePPEncoder, OfflinePPDecoder
from spykshrk.franklab.pp_decoder.visualization import DecodeVisualizer
from spykshrk.franklab.pp_decoder.util import apply_no_anim_boundary

pd.set_option('display.max_rows', 10)
pd.set_option('display.width', 120)

In [2]:
# offline data import for josh
## Define parameters
rat_name = 'remy'
day =  20
epoch = 2

# define data source filepaths
path_base = '/stelmo/anna/'
raw_directory = path_base + rat_name + '/filterframework/'
output_directory = '/stelmo/mcoulter/raw_data/remy/'
day_ep = str(day) + '_' + str(epoch)

# remy real-time
#tetlist = [4,6,9,10,11,12,13,14,15,17,19,20,21,22,23,24,25,26,28,29,30]
tetlist = [24,26]
tetrodes = tetlist
  
pos_bin_size = 5
velocity_thresh_for_enc_dec = 4
velocity_buffer = 0

shift_amt_for_shuffle = 0

use_enc_as_dec = 0  #if you want to use the encoding marks as the decoding marks

discrete_tm_val=.99   # for classifier

print(tetrodes)

[24, 26]


In [3]:
# IMPORT and process data - offline version

#initialize data importer
datasrc = trodes2SS.TrodesImport(raw_directory, rat_name, [day], 
                       [epoch], tetrodes)
# Import marks
marks = datasrc.import_marks()
print('original length: '+str(marks.shape[0]))

# OPTIONAL: to reduce mark number, can filter by size. Current detection threshold is 100  
marks = trodes2SS.threshold_marks(marks, maxthresh=2000,minthresh=150)
# remove any big negative events (artifacts?)
marks = trodes2SS.threshold_marks_negative(marks, negthresh=-999)
print('after filtering: '+str(marks.shape[0]))

# Import trials
trials = datasrc.import_trials()

# Import raw position
linear_pos_raw = datasrc.import_pos(xy='x')   # this is basically just to pull in speed, will be replaced with linearized below
#posY = datasrc.import_pos(xy='y')          #  OPTIONAL; useful for 2d visualization

# if linearization exists, load it. if not, run the linearization.
lin_output1 = output_directory + rat_name + '_' + day_ep + '_' + 'linearized_distance.npy'
lin_output2 = output_directory + rat_name + '_' + day_ep + '_' + 'linearized_track_segments.npy'

print('Linearization found. Loading it')
linear_pos_raw['linpos_flat'] = np.load(lin_output1)   #replace x pos with linerized 
track_segment_ids = np.load(lin_output2)
    
# Import ripples
rips_tmp = datasrc.import_rips(linear_pos_raw, velthresh=4) 
rips = RippleTimes.create_default(rips_tmp,1)  # cast to rippletimes obj
print('Rips less than velocity thresh: '+str(len(rips)))

# generate boundary definitions of each segment
arm_coords, _ = sungod_util.define_segment_coordinates(linear_pos_raw, track_segment_ids)  # optional addition output of all occupied positions (not just bounds)

#bin linear position 
binned_linear_pos, binned_arm_coords, pos_bins = sungod_util.bin_position_data(linear_pos_raw, arm_coords, pos_bin_size)

# for using new linearization nodes
binned_arm_coords[:,1] = 1+binned_arm_coords[:,1]
print(binned_arm_coords)
pos_bin_delta = 1

max_pos = binned_arm_coords[-1][-1]+1

Duplicate marks found (and removed): 0
original length: 640366
0 above 2000uV events removed
382703 below 150uV events removed
0 below -999uV events removed
after filtering: 257663
Linearization found. Loading it
Rips less than velocity thresh: 1256
[  0.   5.  10.  15.  20.  25.  30.  35.  40.  45.  50.  55.  60.  63.
  68.  73.  78.  83.  88.  93.  98. 103. 108. 113. 118. 123. 128. 133.
 138. 141. 146. 151. 156. 161. 166. 171. 176. 181. 186. 191. 196. 201.
 206. 211. 216. 220. 225. 230. 235. 240. 245. 250. 255. 260. 265. 270.
 275. 280. 285. 290. 295. 300. 302. 307. 312. 317. 322. 327. 332. 337.
 342. 347. 352. 357. 362. 367. 372. 377. 382. 387. 392. 397. 402. 407.
 412. 417. 422. 427. 432. 437. 442. 447. 452. 457. 461. 466. 471. 476.
 481. 486. 491. 496. 501. 506. 511. 516. 521. 526. 531. 536. 541. 542.
 547. 552. 557. 562. 567. 572. 577. 582. 587. 592. 597. 602. 607. 612.
 617. 620. 625. 630. 635. 640. 645. 650. 655. 660. 665. 670. 675. 680.
 685.]
[[  0   8]
 [ 13  25]
 [ 29  41]


In [4]:
# merge pos and marks then filter
posinfo_at_mark_times = binned_linear_pos.get_irregular_resampled(marks)
posinfo_at_mark_times = posinfo_at_mark_times.reset_index()
pos_and_mark_movement = posinfo_at_mark_times[(posinfo_at_mark_times['linvel_flat']>4)]
tet26_pos_mark = pos_and_mark_movement[(pos_and_mark_movement['elec_grp_id']==26)]
tet26_pos_mark_trials = tet26_pos_mark[(tet26_pos_mark['time']>1375.22) & (tet26_pos_mark['time']<5483.74)]
tet26_pos_mark_trials

,day,epoch,elec_grp_id,timestamp,time,linpos_flat,linvel_flat
1319,20,2,26,41257198,1375.239933,0,8.205439
1320,20,2,26,41257735,1375.257833,0,7.148180
1323,20,2,26,41258418,1375.280600,0,7.148180
1324,20,2,26,41259383,1375.312767,0,6.162852
1325,20,2,26,41260986,1375.366200,0,4.380185
...,...,...,...,...,...,...,...
257513,20,2,26,164508709,5483.623633,0,6.899070
257515,20,2,26,164509893,5483.663100,0,6.076787
257516,20,2,26,164510452,5483.681733,0,5.322296
257521,20,2,26,164511336,5483.711200,0,4.543361
